In [2]:
import sys
import torch

print("--- Environment Information ---")

# 1. Python Kernel and Version
print(f"Kernel Path: {sys.executable}")
print(f"Python Version: {sys.version}")
print("-" * 30)

# 2. PyTorch and CUDA Status
print("PyTorch Information:")
try:
    print(f"  - PyTorch Version: {torch.__version__}")
    is_cuda_available = torch.cuda.is_available()
    print(f"  - CUDA Available: {is_cuda_available}")
    if is_cuda_available:
        print(f"  - GPU Count: {torch.cuda.device_count()}")
        print(f"  - Current GPU: {torch.cuda.get_device_name(torch.cuda.current_device())}")
    else:
        print("  - Running on CPU.")
except ImportError:
    print("  - PyTorch is NOT installed.")
print("-" * 30)


# 3. Key GeoAI Libraries Check
print("Key Library Versions:")
def check_module_version(module_name):
    try:
        module = __import__(module_name)
        version = getattr(module, '__version__', 'N/A')
        print(f"  - {module_name:<12}: {version}")
    except ImportError:
        print(f"  - {module_name:<12}: NOT INSTALLED")

check_module_version('torchvision')
check_module_version('rasterio')
check_module_version('sklearn')
check_module_version('numpy')
check_module_version('tqdm')

print("\n--- End of Report ---")

--- Environment Information ---
Kernel Path: /home/stellin/miniconda3/bin/python
Python Version: 3.12.2 | packaged by conda-forge | (main, Feb 16 2024, 20:50:58) [GCC 12.3.0]
------------------------------
PyTorch Information:
  - PyTorch Version: 2.6.0+cu124
  - CUDA Available: True
  - GPU Count: 1
  - Current GPU: NVIDIA GeForce RTX 4060 Laptop GPU
------------------------------
Key Library Versions:
  - torchvision : 0.21.0+cu124
  - rasterio    : 1.4.3
  - sklearn     : 1.6.1
  - numpy       : 2.3.1
  - tqdm        : 4.66.4

--- End of Report ---


In [2]:
import os
import torch
import pandas as pd
import json
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from tqdm import tqdm
import time

# --- 1. CONFIGURATION ---
DATA_DIR = 'EuroSAT' 
MODEL_SAVE_PATH = 'landcovernet_resnet18_best.pth'
NUM_CLASSES = 10
BATCH_SIZE = 64
NUM_EPOCHS = 25
LEARNING_RATE = 0.001

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")


# --- 2. CUSTOM DATASET CLASS (No changes) ---
class EuroSATDataset(Dataset):
    def __init__(self, image_paths, labels, class_map, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
        self.class_map = class_map

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label_name = self.labels[idx]
        
        image = Image.open(image_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        label_idx = self.class_map[label_name]
        return image, label_idx

# --- 3. DATA PREPARATION (Corrected Section) ---

print("Loading data from CSV files...")

# Load the class name to index mapping
with open(os.path.join(DATA_DIR, 'label_map.json'), 'r') as f:
    class_map = json.load(f)  # <<< FIX 1: Passed the file object 'f' to the function.

class_names = [k for k, v in sorted(class_map.items(), key=lambda item: item[1])]

def load_data_from_csv(csv_path, base_dir):
    """Reads a CSV file and returns lists of full image paths and labels."""
    # We skip the first column as it seems to be an unnamed index
    df = pd.read_csv(csv_path, usecols=['Filename', 'ClassName'])
    
    # <<< FIX 2: Using the correct, case-sensitive column names from your CSV.
    full_paths = [os.path.join(base_dir, fname) for fname in df['Filename']]
    labels = df['ClassName'].tolist()
    return full_paths, labels

# Construct paths to the CSV files
train_csv_path = os.path.join(DATA_DIR, 'train.csv')
val_csv_path = os.path.join(DATA_DIR, 'validation.csv')
test_csv_path = os.path.join(DATA_DIR, 'test.csv')

# Load the file paths and labels for each split
train_paths, train_labels = load_data_from_csv(train_csv_path, DATA_DIR)
val_paths, val_labels = load_data_from_csv(val_csv_path, DATA_DIR)
test_paths, test_labels = load_data_from_csv(test_csv_path, DATA_DIR)


print(f"Training samples: {len(train_paths)}")
print(f"Validation samples: {len(val_paths)}")
print(f"Test samples: {len(test_paths)}")
print("\nClass Mapping Loaded:")
print(class_map)
print("")


# --- 4. DATA TRANSFORMATION & LOADING (No changes) ---

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

train_dataset = EuroSATDataset(train_paths, train_labels, class_map, transform=data_transforms['train'])
val_dataset = EuroSATDataset(val_paths, val_labels, class_map, transform=data_transforms['val'])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)


# --- 5. MODEL SETUP (No changes) ---

model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, NUM_CLASSES)
model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)


# --- 6. TRAINING AND VALIDATION LOOP (No changes) ---

best_val_acc = 0.0
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}")
    print('-' * 10)

    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
            dataloader = train_loader
        else:
            model.eval()
            dataloader = val_loader

        running_loss = 0.0
        running_corrects = 0

        progress_bar = tqdm(dataloader, desc=f"{phase.capitalize()} Phase")
        for inputs, labels in progress_bar:
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            
            progress_bar.set_postfix(loss=loss.item())

        epoch_loss = running_loss / len(dataloader.dataset)
        epoch_acc = running_corrects.double() / len(dataloader.dataset)

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        if phase == 'val' and epoch_acc > best_val_acc:
            best_val_acc = epoch_acc
            torch.save(model.state_dict(), MODEL_SAVE_PATH)
            print(f"New best model saved to {MODEL_SAVE_PATH} with accuracy: {best_val_acc:.4f}")

    print()

end_time = time.time()
total_time = end_time - start_time
print(f"Training complete in {total_time // 60:.0f}m {total_time % 60:.0f}s")
print(f"Best Val Acc: {best_val_acc:4f}")

Using device: cuda
Loading data from CSV files...
Training samples: 18900
Validation samples: 5400
Test samples: 2700

Class Mapping Loaded:
{'AnnualCrop': 0, 'Forest': 1, 'HerbaceousVegetation': 2, 'Highway': 3, 'Industrial': 4, 'Pasture': 5, 'PermanentCrop': 6, 'Residential': 7, 'River': 8, 'SeaLake': 9}



/home/stellin/miniconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/stellin/miniconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/25
----------


Train Phase: 100%|██████████| 296/296 [00:42<00:00,  7.00it/s, loss=0.761]


train Loss: 0.5952 Acc: 0.8066


...

val Loss: 0.1224 Acc: 0.9581

Epoch 25/25
----------


Train Phase: 100%|██████████| 296/296 [00:37<00:00,  7.85it/s, loss=0.155] 


train Loss: 0.1845 Acc: 0.9381


Val Phase: 100%|██████████| 85/85 [00:03<00:00, 23.97it/s, loss=0.132] 

val Loss: 0.1116 Acc: 0.9611

Training complete in 18m 11s
Best Val Acc: 0.969444
